Программа для поиска самого выгодного арбитража на биржах криптовалют

In [ ]:
!pip install ccxt

In [ ]:
import ccxt

In [ ]:
import pandas as pd
import numpy as np
import datetime

In [ ]:
# название рынка и название символа
def get_candles(market, symbol):
  import ccxt
  exchange = getattr(ccxt, market)({ 'enableRateLimit': True })
  candles = exchange.fetch_ohlcv(symbol, '1m')
  df1 = pd.DataFrame(candles, columns=['ts', symbol+'-open', symbol+'-high', symbol+'-low', symbol+'-close', 'volume'])
  return df1

In [ ]:
# На вход два df и коэффициент сравнения
def compare_pairs(df1, df2, coeff, symbol):  
  df = df1.merge(df2, on='ts')
  res = pd.DataFrame()
  res = df[['ts', symbol+'-close_x', symbol+'-close_y']][abs(df[symbol+'-close_x']  - df[symbol+'-close_y']) > (0.0001 * df[symbol+'-close_x'])]
  res['profit1'] = (abs(res[symbol+'-close_x'] - res[symbol+'-close_y'])) / res[symbol+'-close_x'] 
  res['profit2'] = (abs(res[symbol+'-close_x'] - res[symbol+'-close_y'])) / res[symbol+'-close_y'] 
  res = res[(res['profit1'] > coeff) | (res['profit2'] > coeff)]
  res['profit'] = np.where(res.profit1 > res.profit2, abs(res[symbol+'-close_x']) * (res.profit1 - coeff),  abs(res[symbol+'-close_y']) * (res.profit2 - coeff))
  res['ts'] = pd.to_datetime(res['ts'],unit='ms')
  utc = 18000 
  res.ts = res.ts + datetime.timedelta(0, utc)
  result = res[['ts', symbol+'-close_x', symbol+'-close_y', 'profit']]
  sexy = 0
  if len(result) > 0:
    print(result)
    prof = round(sum(result.profit), 4)
    sexy = prof / len(result)
    print('Прибыль:', prof, symbol)
    print('Прибыль / кол-во сделок:', str(sexy))   
    
  else: 
    print('No profit')
  
  return sexy

In [ ]:
def find_profit(market1, market2, symbol, coeff):
  df1 = get_candles(market1, symbol)  
  df2 = get_candles(market2, symbol)  
  return compare_pairs(df1, df2, coeff, symbol)

In [ ]:
symbols = ['ETH/BTC', 'ETH/EUR', 'XRP/USDT', 'LTC/USDT', 'ETH/USDT']

In [ ]:
markets = ['kraken', 'binance', 'ftx']

In [ ]:
best = 0
top_pair = []
for i in range(len(markets)-1):
  market1 = markets[i]
  for j in range(i+1, len(markets)):
    market2 = markets[j]     
    print(market1, market2) 
    for s in symbols:
      print(s)
      check = find_profit(market1, market2, s, 0.002)
      if check > best:
        best = check
        top_pair = [market1, market2, s]
print('Сегодня настроим бота на рынки: ' + str(top_pair[0]), str(top_pair[1]) +' на пару '+str(top_pair[2]))  


kraken binance
ETH/BTC
No profit
ETH/EUR
                     ts  ETH/EUR-close_x  ETH/EUR-close_y   profit
24  2021-04-01 17:01:00          1634.03          1637.65  0.35194
404 2021-04-01 23:21:00          1615.00          1618.39  0.16000
Прибыль: 0.5119 ETH/EUR
Прибыль / кол-во сделок: 0.25595
XRP/USDT
                     ts  XRP/USDT-close_x  XRP/USDT-close_y    profit
9   2021-04-01 16:47:00           0.55852           0.55704  0.000366
11  2021-04-01 16:49:00           0.55708           0.55842  0.000226
19  2021-04-01 16:57:00           0.55704           0.55860  0.000446
20  2021-04-01 16:58:00           0.55852           0.55732  0.000085
22  2021-04-01 17:00:00           0.55585           0.55404  0.000702
..                  ...               ...               ...       ...
467 2021-04-02 00:25:00           0.56581           0.56376  0.000922
468 2021-04-02 00:26:00           0.56581           0.56393  0.000752
469 2021-04-02 00:27:00           0.56581           0.56393  0